In [80]:
%matplotlib inline

import matplotlib.pyplot as plt
from scipy.io import loadmat
import numpy as np
from tqdm import tqdm
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [3]:
X_raw = loadmat("R2198_20ms.mat")
y_raw = np.loadtxt("R2198_locations.dat")

X_raw = X_raw['mm'].T

In [108]:
def generate_datasets(n, fft):
    print(n, 'fft:', fft)
    X = np.zeros((X_raw.shape[0],2*n*33))
    y = np.zeros(y_raw.shape)
#     for i in tqdm(range(n, X_raw.shape[0]-n)):
    for i in range(n, X_raw.shape[0]-n):
        if fft:
            X[i] = np.real(np.fft.fft(X_raw[i-n:i+n].T)).flatten()
        else:
            X[i] = X_raw[i-n:i+n].flatten(order='F')
        y[i] = y_raw[i]

    X = X[n:-n]
    y = y[n:-n]

#     X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
    return train_test_split(X, y, test_size=0.2, random_state=0)

In [110]:
results = []
models = [RandomForestRegressor(n_jobs=-1, verbose=0), 
            ExtraTreesRegressor(n_jobs=-1, verbose=0), 
            LinearRegression(n_jobs=-1)]

time_window = 10
for time_window in [10,20,50,75]:
# for time_window in [85]:

    use_fft = False
    X_train, X_test, y_train, y_test = generate_datasets(time_window, fft=use_fft)

    for clf in models:
        clf.fit(X_train, y_train)
        results.append({'time_window': time_window,
                        'fft': use_fft, 
                        'MAE': mean_absolute_error(clf.predict(X_test), y_test),
                        'model': clf.__class__.__name__})
        print(results)

10 fft: False
[{'model': 'RandomForestRegressor', 'time_window': 10, 'MAE': 14.839382530709283, 'fft': False}]
[{'model': 'RandomForestRegressor', 'time_window': 10, 'MAE': 14.839382530709283, 'fft': False}, {'model': 'ExtraTreesRegressor', 'time_window': 10, 'MAE': 15.151198991883073, 'fft': False}]
[{'model': 'RandomForestRegressor', 'time_window': 10, 'MAE': 14.839382530709283, 'fft': False}, {'model': 'ExtraTreesRegressor', 'time_window': 10, 'MAE': 15.151198991883073, 'fft': False}, {'model': 'LinearRegression', 'time_window': 10, 'MAE': 15.835587966171499, 'fft': False}]
20 fft: False
[{'model': 'RandomForestRegressor', 'time_window': 10, 'MAE': 14.839382530709283, 'fft': False}, {'model': 'ExtraTreesRegressor', 'time_window': 10, 'MAE': 15.151198991883073, 'fft': False}, {'model': 'LinearRegression', 'time_window': 10, 'MAE': 15.835587966171499, 'fft': False}, {'model': 'RandomForestRegressor', 'time_window': 20, 'MAE': 13.95617703658775, 'fft': False}]
[{'model': 'RandomForestR